# Proyecto Final: Análisis de Rendimiento de Dotplot Secuencial vs. Paralelización 

descripcion del proyecto

## Código base para leer archivos FASTA

In [4]:
from Bio import SeqIO

def cargar_secuencias_fasta(file1, file2):
    def merge_sequences(file_path):
        sequences = []
        for record in SeqIO.parse(file_path, "fasta"):
            sequences.append(str(record.seq))
        return "".join(sequences)

    secuencia1 = merge_sequences(file1)
    secuencia2 = merge_sequences(file2)
    return secuencia1, secuencia2

## Versión Secuencial para el Dotplot

In [5]:
import numpy as np
import matplotlib.pyplot as plt

def dotplot_secuencial(secuencia1, secuencia2):
    dotplot = np.zeros((len(secuencia1), len(secuencia2)))

    for i in range(len(secuencia1)):
        for j in range(len(secuencia2)):
            dotplot[i, j] = 1 if secuencia1[i] == secuencia2[j] else 0

    return dotplot

def generar_dotplot(dotplot, nombre_salida):
    plt.figure(figsize=(10, 10))
    plt.imshow(dotplot, cmap='Greys', aspect='auto')
    plt.xlabel("Secuencia 2")
    plt.ylabel("Secuencia 1")
    plt.savefig(nombre_salida)
    plt.close()

## Versión con Multiprocessing

In [6]:
import multiprocessing as mp

def comparar_indices(args):
    i, secuencia1, secuencia2 = args
    return [1 if secuencia1[i] == secuencia2[j] else 0 for j in range(len(secuencia2))]

def dotplot_multiprocessing(secuencia1, secuencia2, num_procesos=mp.cpu_count()):
    with mp.Pool(num_procesos) as pool:
        resultados = pool.map(
            comparar_indices, [(i, secuencia1, secuencia2) for i in range(len(secuencia1))]
        )

    dotplot = np.array(resultados)
    return dotplot

## Versión Paralela con mpi4py

In [7]:
from mpi4py import MPI

def dotplot_mpi(secuencia1, secuencia2):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    chunk_size = len(secuencia1) // size
    inicio = rank * chunk_size
    fin = len(secuencia1) if rank == size - 1 else (rank + 1) * chunk_size

    dotplot_local = np.zeros((fin - inicio, len(secuencia2)))

    for i in range(inicio, fin):
        for j in range(len(secuencia2)):
            dotplot_local[i - inicio, j] = 1 if secuencia1[i] == secuencia2[j] else 0

    dotplot = None
    if rank == 0:
        dotplot = np.zeros((len(secuencia1), len(secuencia2)))

    comm.Gather(dotplot_local, dotplot, root=0)

    if rank == 0:
        return dotplot
    return None

## Versión Paralela con pyCuda

In [8]:
import pycuda.autoinit
import pycuda.gpuarray as gpuarray
from pycuda.compiler import SourceModule
import numpy as np

mod = SourceModule("""
__global__ void generar_dotplot(char *sec1, char *sec2, int *dotplot, int len1, int len2) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i < len1 && j < len2) {
        dotplot[i * len2 + j] = (sec1[i] == sec2[j]) ? 1 : 0;
    }
}
""")

def dotplot_pycuda(secuencia1, secuencia2):
    sec1_gpu = gpuarray.to_gpu(np.array(list(secuencia1), dtype=np.byte))
    sec2_gpu = gpuarray.to_gpu(np.array(list(secuencia2), dtype=np.byte))
    dotplot_gpu = gpuarray.zeros((len(secuencia1), len(secuencia2)), dtype=np.int32)

    block_size = (16, 16, 1)
    grid_size = (
        (len(secuencia1) + block_size[0] - 1) // block_size[0],
        (len(secuencia2) + block_size[1] - 1) // block_size[1],
    )

    func = mod.get_function("generar_dotplot")
    func(
        sec1_gpu, sec2_gpu, dotplot_gpu,
        np.int32(len(secuencia1)), np.int32(len(secuencia2)),
        block=block_size, grid=grid_size,
    )

    return dotplot_gpu.get()

In [9]:
import pycuda.autoinit
import pycuda.driver as cuda
print(f"CUDA driver version: {cuda.get_version()}")

CUDA driver version: (12, 6, 0)


## Función de Filtrado para Detectar Líneas Diagonales

In [10]:
def filtrar_diagonales(dotplot, umbral=5):
    diagonales = []
    for d in range(-dotplot.shape[0] + 1, dotplot.shape[1]):
        diagonal = np.diag(dotplot, k=d)
        if np.sum(diagonal) >= umbral:
            diagonales.append((d, diagonal))
    return diagonales

## Aplicación de Línea de Comandos

In [ ]:
import argparse

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Generar un dotplot de dos secuencias.")
    parser.add_argument("--file1", required=True, help="Archivo FASTA 1.")
    parser.add_argument("--file2", required=True, help="Archivo FASTA 2.")
    parser.add_argument("--output", required=True, help="Nombre del archivo de salida.")
    parser.add_argument("--mode", choices=["secuencial", "multiprocessing", "mpi", "pycuda"],
                        required=True, help="Modo de ejecución.")

    args = parser.parse_args()

    secuencia1, secuencia2 = cargar_secuencias_fasta(args.file1, args.file2)

    if args.mode == "secuencial":
        dotplot = dotplot_secuencial(secuencia1, secuencia2)
    elif args.mode == "multiprocessing":
        dotplot = dotplot_multiprocessing(secuencia1, secuencia2)
    elif args.mode == "mpi":
        dotplot = dotplot_mpi(secuencia1, secuencia2)
    elif args.mode == "pycuda":
        dotplot = dotplot_pycuda(secuencia1, secuencia2)

    generar_dotplot(dotplot, args.output)